In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
node_path = '../data/processed_data/node_link/daegu_selected_nodes.csv'
link_path = '../data/processed_data/node_link/daegu_selected_links.csv'

node = pd.read_csv(node_path)
link = pd.read_csv(link_path)

In [3]:
def fixer(x):
    try: return int(x)
    except ValueError:
        return 0
    
link_set = set(link['LINK_ID'])
target_file_list = [f'202309{i:02}_5Min.csv' for i in range(1, 31)]
    
for target_file in target_file_list:
    traffic_data_path = '../data/raw_data/traffic/' + target_file
    traffic_data_col = ['Date', 'Time', 'LINK_ID', 'Speed']
    traffic_data_type = {'Date':'Int32', 'Time':'Int32', 'LINK_ID':'Int64','Speed':'Float32'}

    traffic_index = pd.read_csv(traffic_data_path, usecols=[2], names=['LINK_ID'], converters={2:fixer}, on_bad_lines='warn')    
    
    temp = traffic_index[traffic_index['LINK_ID'].map(lambda x: x not in link_set)].index
    traffic_data = pd.read_csv(traffic_data_path, usecols=[0,1,2,4], names=traffic_data_col, dtype=traffic_data_type, skiprows=temp)
    traffic_data = traffic_data[['Time', 'LINK_ID', 'Speed']].set_index(['LINK_ID', 'Time']).unstack(level=0)
    traffic_data.columns = [x[1] for x in traffic_data.columns.values]
    traffic_data = traffic_data.sort_index()
    
    print(f'{target_file}: {traffic_data.isna().sum().sum()}')
    if traffic_data.isna().sum().sum() > 0:
        print('Fill na to 0')
        traffic_data.fillna(value=0)
    print(f'{target_file}: {traffic_data.isna().sum().sum()}')
    print((link['LINK_ID'] == traffic_data.columns.values).sum() == len(link['LINK_ID']))
    
    l = len(traffic_data.index)
    prev = traffic_data.index[0]
    append_list = []
    idx_list = []
    pos = 1

    while pos < l:
        cur = traffic_data.index[pos]
        gap = (cur//100-prev//100)*60+cur%100-prev%100
        for i in range(5, gap, 5):
            prev += 5
            if prev % 100 == 60:
                prev += 40
            append_list.append(((gap-i)*traffic_data.iloc[pos-1]+i*traffic_data.iloc[pos])/gap)
            idx_list.append(prev)
        pos += 1
        prev = cur
    
    traffic_data = pd.concat([traffic_data, pd.DataFrame(append_list, index=idx_list)]).sort_index()
    print(len(traffic_data))
    traffic_data.to_csv(os.path.join('../data/processed_data/traffic', 'p_'+target_file), index=False) 

20231001_5Min.csv: 0
20231001_5Min.csv: 0
True
287
20231002_5Min.csv: 0
20231002_5Min.csv: 0
True
288
20231003_5Min.csv: 0
20231003_5Min.csv: 0
True
288
20231004_5Min.csv: 0
20231004_5Min.csv: 0
True
288
20231005_5Min.csv: 0
20231005_5Min.csv: 0
True
288
20231006_5Min.csv: 0
20231006_5Min.csv: 0
True
288
20231007_5Min.csv: 0
20231007_5Min.csv: 0
True
288
20231008_5Min.csv: 0
20231008_5Min.csv: 0
True
288
20231009_5Min.csv: 0
20231009_5Min.csv: 0
True
288
20231010_5Min.csv: 0
20231010_5Min.csv: 0
True
288
20231011_5Min.csv: 0
20231011_5Min.csv: 0
True
288
20231012_5Min.csv: 0
20231012_5Min.csv: 0
True
288
20231013_5Min.csv: 0
20231013_5Min.csv: 0
True
288
20231014_5Min.csv: 0
20231014_5Min.csv: 0
True
288
20231015_5Min.csv: 0
20231015_5Min.csv: 0
True
288
20231016_5Min.csv: 0
20231016_5Min.csv: 0
True
288
20231017_5Min.csv: 0
20231017_5Min.csv: 0
True
288
20231018_5Min.csv: 0
20231018_5Min.csv: 0
True
288
20231019_5Min.csv: 0
20231019_5Min.csv: 0
True
288
20231020_5Min.csv: 0
20231020_5

# Concat data

In [4]:
final_data_list = [f'../data/processed_data/traffic/p_202309{i:02}_5Min.csv' for i in range(1, 31)]
final_data = pd.concat([pd.read_csv(path) for path in final_data_list])
final_data.to_csv('../data/processed_data/traffic/202309.csv', index=False) 